Finetune on IMDb database using AdamW optimizer

# Necessary Libraries

In [1]:
%pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 785.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Load Dataset and Model

In [40]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [37]:
model_name = "distilbert-base-uncased"

student_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
from datasets import load_dataset

dataset = load_dataset("imdb")

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

train_dataset = dataset["train"].map(tokenize_function, batched=True)
test_dataset = dataset["test"].map(tokenize_function, batched=True)



Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

# Evaluation Metrics

In [43]:
import evaluate
import numpy as np

def metrics(eval_pred):
    metric = evaluate.load("accuracy")

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Train Model

In [44]:
training_args = TrainingArguments(
    output_dir="distilbert-finetuned-imdb",          # Directory to save the model checkpoints
    eval_strategy="epoch",     # Evaluate after each epoch
    save_strategy="epoch",           # Save model at each epoch
    learning_rate=5e-5,              # Learning rate
    per_device_train_batch_size=16,  # Speed up training
    per_device_eval_batch_size=16,   # Speed up evaluation
    num_train_epochs=1,              # Reduce training time
#    weight_decay=0.01,               # Weight decay (L2 regularization)
    logging_steps=10,                # Log every 10 steps
    load_best_model_at_end=True,     # Load best model at the end of training
    fp16=True,                       # Use mixed precision for faster training on GPU
    report_to="none",                 # Disable WandB logging
    optim="adamw_torch",              # Use AdamW optimizer
    lr_scheduler_type="linear"       # Use linear learning rate scheduler
)

In [45]:
from transformers import DataCollatorWithPadding

collator = DataCollatorWithPadding(tokenizer=tokenizer) # Dynamic padding

trainer = Trainer(
    model=student_model,
    args=training_args,
    data_collator=collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.257300,0.190451,0.929800


TrainOutput(global_step=1563, training_loss=0.25673115352598885, metrics={'train_runtime': 443.8732, 'train_samples_per_second': 56.322, 'train_steps_per_second': 3.521, 'total_flos': 3280166004732288.0, 'train_loss': 0.25673115352598885, 'epoch': 1.0})

# Upload model to Hugging Face Hub

In [46]:
from huggingface_hub import notebook_login

notebook_login() # Login to account

In [47]:
kwargs = {
    "dataset_tags": "imdb",
    "dataset": "imdb",
    "model_name": f"{model_name}-finetuned-imdb",
    "finetuned_from": model_name,
    "tasks": "text-classification"
}

trainer.push_to_hub(commit_message="Using different optimizer and learning rate scheduler to improve performance", **kwargs)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/avanishd/distilbert-finetuned-imdb/commit/ff5518bd2e8e66968651e6f3e73f7010d653e514', commit_message='Using different optimizer and learning rate scheduler to improve performance', commit_description='', oid='ff5518bd2e8e66968651e6f3e73f7010d653e514', pr_url=None, repo_url=RepoUrl('https://huggingface.co/avanishd/distilbert-finetuned-imdb', endpoint='https://huggingface.co', repo_type='model', repo_id='avanishd/distilbert-finetuned-imdb'), pr_revision=None, pr_num=None)